In [ ]:
import pandas as pd


data = pd.read_csv(
    "dane_lista_1.csv",
    dtype={"line": str},
)

start_stops = set(data["start_stop"])
end_stops = set(data["end_stop"])
start_stops.difference(end_stops)
data.columns

996520


##### Uważka
Są przejazdy gdzie przystanek początkowy === przystanek końcowy, ale przy dobrze zaimplementowanym algorytmie, nie powinno mieć to wpływu, więc nie ma potrzeby usuwania tych przejazdów.

In [ ]:

# Przy załadowaniu wszystkich danych poniższy kod wypluje ponad 13k wierszy
data[data["start_stop"] == data["end_stop"]]

### Uproszczenie danych związanych z lokalizacją
Jeden przystanek może mieć wiele współrzędnych geograficznych.
Dla uproszczenia zadania obliczamy tylko jedną wartość dla jednego przystanku (np. wyliczenie średniej ze wszystkich współrzędnych związanych z danym przytankiem)

Struktura agregująca nazwę przystanku i współrzędne:
Mapa:
- klucz - nazwa przystanku (string)
- wartość - zbiór (set) krotek o strukturze (szerokość geo, długość geo)
    - pytanie czy nie lepiej trzymać listę jako wartość - pojawią się duplikaty, więc przy liczeniu średniej wpłyną one na końcowe położenie przystanku - czy będzie to bliższe rzeczywistości?

### Zbiór jako wartość

In [ ]:
stops_map_set: dict[str, set[tuple[float, float]]] = {}
for start, start_lat, start_lon, end, end_lat, end_lon in zip(
    data.start_stop,
    data.start_stop_lat,
    data.start_stop_lon,
    data.end_stop,
    data.end_stop_lat,
    data.end_stop_lon,
):
    stops_map_keys = stops_map_set.keys()
    if start in stops_map_keys:
        stops_map_set[start].add((start_lat, start_lon))
    else:
        stops_map_set[start] = {(start_lat, start_lon)}

    if end in stops_map_keys:
        stops_map_set[end].add((end_lat, end_lon))
    else:
        stops_map_set[end] = {(end_lat, end_lon)}

print(f"Number of stops: {len(stops_map_set)}")
print("Stops and their coordinates:")
stops_map_set


In [ ]:
final_stops_coords_set: dict[str, tuple[float, float]] = {}
for stop, coordinates in stops_map_set.items():
    coordinates_size = len(coordinates)
    avg_lat = 0
    avg_lon = 0
    for lat, lon in coordinates:
        avg_lat += lat
        avg_lon += lon

    avg_lat = avg_lat/coordinates_size
    avg_lon = avg_lon/coordinates_size
    final_stops_coords_set[stop] = (avg_lat, avg_lon)

print(f"Number of final stops: {len(final_stops_coords_set)}")
print("Stops and their final coordinates:")
final_stops_coords_set

In [ ]:
final_stops_coords_set["Komuny Paryskiej"]


### Lista jako wartość

In [ ]:
stops_map_list: dict[str, list[tuple[float, float]]] = {}
for start, start_lat, start_lon, end, end_lat, end_lon in zip(
    data.start_stop,
    data.start_stop_lat,
    data.start_stop_lon,
    data.end_stop,
    data.end_stop_lat,
    data.end_stop_lon,
):
    stops_map_keys = stops_map_list.keys()
    if start in stops_map_keys:
        stops_map_list[start].append((start_lat, start_lon))
    else:
        stops_map_list[start] = [(start_lat, start_lon)]

    if end in stops_map_keys:
        stops_map_list[end].append((end_lat, end_lon))
    else:
        stops_map_list[end] = [(end_lat, end_lon)]

print(f"Number of stops: {len(stops_map_set)}")
print("Stops and their coordinates:")
stops_map_list

In [ ]:
final_stops_coords_list: dict[str, tuple[float, float]] = {}
for stop, coordinates in stops_map_list.items():
    coordinates_size = len(coordinates)
    avg_lat = 0
    avg_lon = 0
    for lat, lon in coordinates:
        avg_lat += lat
        avg_lon += lon

    avg_lat = avg_lat/coordinates_size
    avg_lon = avg_lon/coordinates_size
    final_stops_coords_list[stop] = (avg_lat, avg_lon)

print(f"Number of final stops: {len(final_stops_coords_list)}")
print("Stops and their final coordinates:")
final_stops_coords_list

In [ ]:
print(f"Lokalizacja przystanku wyliczona ze zbioru: {final_stops_coords_set["Komuny Paryskiej"]}")
print(f"Lokalizacja przystanku wyliczona z listy: {final_stops_coords_list["Komuny Paryskiej"]}")


### Wniosek
Wyliczanie średniej lokalizacji przystanku na podstawie wartości, które mogą się duplikować wydaje się być lepszym podejściem - ostateczna lokalizacja dokładniej przybliża hipotetyczne położenie przystanku.

Zapisanie nowych lokalizacji (średnich) do ramki

In [ ]:
updated_data = data.copy()
    
# Replace start stop coordinates
updated_data['start_stop_lat'] = updated_data['start_stop'].map(lambda x: final_stops_coords_list[x][0])
updated_data['start_stop_lon'] = updated_data['start_stop'].map(lambda x: final_stops_coords_list[x][1])
    
# Replace end stop coordinates
updated_data['end_stop_lat'] = updated_data['end_stop'].map(lambda x: final_stops_coords_list[x][0])
updated_data['end_stop_lon'] = updated_data['end_stop'].map(lambda x: final_stops_coords_list[x][1])

updated_data[(updated_data['start_stop'] == 'PL. GRUNWALDZKI') & (updated_data['end_stop'] == 'PL. GRUNWALDZKI')]
    

Godziny są spoza zakresu 0-24. **Jak to obsłużyć???**

Minimalna wartość godziny to **3** <br>
Maksymalna wartość godziny to **30**

In [ ]:
import re
from datetime import datetime, timedelta

regexp = re.compile(r"(\d{2}):(\d{2}):(\d{2})")

for start_time, end_time in zip(updated_data.departure_time, updated_data.arrival_time):
    for t in [start_time,end_time]:
        match = re.search(regexp, t)
        if match is None:
            raise Exception("Shouldn't happen")
        h, m, s = map(int, match.groups())
        if h >= 24:
            #print(t)
            h -= 24
            dt = datetime.strptime(f"{h}:{m}:{s}", "%H:%M:%S")
            dt += timedelta(1)
            #print(dt)
        else:
            dt = pd.to_datetime(f"{h}:{m}:{s}", format="%H:%M:%S")
            #print(dt)

Zapisanie nowych danych do pliku

In [ ]:
updated_data.to_csv('./data_with_normalized_locations.csv')